In [1]:
#import packages needed

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')
import numpy as np
import json
import requests
from bs4 import BeautifulSoup
from re import sub
from decimal import Decimal
%matplotlib inline

## Box Office Mojo

Marvel Movies URL: https://www.boxofficemojo.com/brand/bn3732077058/?ref_=bo_bns_table_1

In [3]:
marvel_url = 'https://www.boxofficemojo.com/brand/bn3732077058/?ref_=bo_bns_table_1'

In [22]:
#BeautifulSouping the url's for Marvel movies from Movie Mojo

def soupitupMarvel(url, genre_string):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    titles = soup.find_all('td', class_='a-text-left mojo-field-type-release mojo-cell-wide')
    lifetime_gross = soup.find_all('td', class_='a-text-right mojo-field-type-money mojo-sort-column mojo-estimatable')
    date_release = soup.find_all('td', class_='a-text-left mojo-field-type-date a-nowrap')
    
    Genre_List =[]
    cur=0
    while cur < len(titles):
        Genre_Dict = {}
        
        Genre_Dict['Date_Released'] = date_release[cur].get_text()
        Genre_Dict['Lifetime_Gross'] = lifetime_gross[cur].get_text()
        Genre_Dict['Title'] = titles[cur].get_text()
        
        Genre_List.append(Genre_Dict)
        cur+=1
    
    genre = pd.DataFrame(Genre_List)
    genre['Subgenre'] = genre_string
    return genre

In [24]:
marvel_df = soupitupMarvel(marvel_url, 'Marvel')

In [31]:
#Converting the Lifetime_Gross column into type decimal and then convert it again to int

marvel_df['Lifetime_Gross'] = marvel_df['Lifetime_Gross'].apply(lambda x: Decimal(sub(r'[^\d.]', '', x)))
marvel_df['Lifetime_Gross'] = marvel_df['Lifetime_Gross'].apply(lambda x: int(x))

In [37]:
print('Meadian: $',round(marvel_df['Lifetime_Gross'].median()/1000000), 'M')
print('Mean: $',round(marvel_df['Lifetime_Gross'].mean()/1000000), 'M')

Meadian: $ 210 M
Mean: $ 237 M


In [25]:
marvel_df.to_csv('marveldf.csv')

In [19]:
#correlation analysis: do people watch marvel movies because they're marvel or because they're in the supernatural genre? 